In [1]:
import os
import re
from glob import glob

In [3]:
files = glob('./data/CBData/cbtest_*.txt')
files

['./data/CBData/cbtest_NE_valid_2000ex.txt',
 './data/CBData/cbtest_NE_train.txt',
 './data/CBData/cbtest_V_train.txt',
 './data/CBData/cbtest_CN_train.txt',
 './data/CBData/cbtest_NE_test_2500ex.txt',
 './data/CBData/cbtest_V_valid_2000ex.txt',
 './data/CBData/cbtest_P_test_2500ex.txt',
 './data/CBData/cbtest_V_test_2500ex.txt',
 './data/CBData/cbtest_P_train.txt',
 './data/CBData/cbtest_CN_test_2500ex.txt',
 './data/CBData/cbtest_CN_valid_2000ex.txt',
 './data/CBData/cbtest_P_valid_2000ex.txt']

In [4]:
word2idx = {}
word2idx[''] = 0
word2idx['.'] = 1
word2idx[','] = 2
word2idx['?'] = 3
word2idx["''"] = 4
word2idx['|'] = 5
word2idx['<UKN>'] = 6
for file in files:
    if 'P_train' in file:
        continue
    with open(file, 'r', encoding='utf8') as f:
        lines = f.readlines()
    print('there are %d examples in %s' % (len(lines) / 22, file))
    all_words = [word for line in lines for word in re.split(' |\n|\t|\|', line)]
    
    for word in all_words:
        if word not in word2idx:
            word2idx[word] = len(word2idx)

there are 2000 examples in ./data/CBData/cbtest_NE_valid_2000ex.txt
there are 108719 examples in ./data/CBData/cbtest_NE_train.txt
there are 105825 examples in ./data/CBData/cbtest_V_train.txt
there are 120769 examples in ./data/CBData/cbtest_CN_train.txt
there are 2500 examples in ./data/CBData/cbtest_NE_test_2500ex.txt
there are 2000 examples in ./data/CBData/cbtest_V_valid_2000ex.txt
there are 2500 examples in ./data/CBData/cbtest_P_test_2500ex.txt
there are 2500 examples in ./data/CBData/cbtest_V_test_2500ex.txt
there are 2500 examples in ./data/CBData/cbtest_CN_test_2500ex.txt
there are 2000 examples in ./data/CBData/cbtest_CN_valid_2000ex.txt
there are 2000 examples in ./data/CBData/cbtest_P_valid_2000ex.txt


In [5]:
len(word2idx)

63287

In [6]:
file = './data/CBData/cbtest_P_train.txt'
with open(file, 'r', encoding='utf8') as f:
    lines = f.readlines()
print('there are %d examples in %s' % (len(lines) / 22, file))
# all_words = [word for line in lines for word in re.split(' |\n|\t|\|', line)]

# for word in all_words:
#     if word not in word2idx:
#         word2idx[word] = len(word2idx)

there are 334030 examples in ./data/CBData/cbtest_P_train.txt


In [10]:
total = len(lines)
split = total // 4
split

1837165

In [14]:
for sp in range(4):
    start = split * sp
    end = split * (sp + 1) if sp < 3 else total
    all_words = [word for line in lines[start:end] for word in re.split(' |\n|\t|\|', line)]
    for word in all_words:
        if word not in word2idx:
            word2idx[word] = len(word2idx)

In [15]:
len(word2idx)

63293

In [16]:
import pickle
with open('./processed/word2idx.pkl', 'wb') as f:
    pickle.dump(word2idx, f)

In [25]:
def read_our_data(fname, count, word2idx):
    if os.path.isfile(fname):
        with open(fname, encoding='utf8') as f:
            lines = f.readlines()
    else:
        raise FileNotFoundError("[!] Data %s not found" % fname)

    assert(len(lines) % 22 == 0)
    
    # Test code! need to be deleted when actually using
#     lines = lines[:22*30000]

    n_question = len(lines) // 22
    print('Number of example in %s: %d' % (fname, n_question))

    contexts = []
    querys = []
    candidates = []
    answers = []
    for i in range(n_question):
        if i % 10000 == 0:
            print('Processing %d' % i)
        hey = lines[i * 22: (i+1) * 22]
        context = hey[:20]
        context = [[word2idx[word] for word in sen.split()[1:]] for sen in context]
        contexts.append(context)

        last = hey[20]
        last = last.split('\t')
        querys.append([word2idx[word] for word in last[0].split()[1:]])
        answers.append(word2idx[last[1]])
        # last[2] is a empty string
        candidates.append([word2idx[word] for word in last[3][:-1].split('|')])
        

    # TODO: Discard too long context
    # TODO: How to encode query?

    return {'contexts': contexts,
            'querys': querys,
            'candidates': candidates,
            'answers': answers}

In [33]:
file = files[0]
file[14:-4]

'cbtest_NE_valid_2000ex'

In [37]:
for file in files:
    data = read_our_data(file, [], word2idx)
    with open('./processed/%s.pkl' % file[14:-4], 'wb') as f:
        pickle.dump(data, f)

Number of example in ./data/CBData/cbtest_NE_valid_2000ex.txt: 2000
Processing 0
Number of example in ./data/CBData/cbtest_NE_train.txt: 108719
Processing 0
Processing 10000
Processing 20000
Processing 30000
Processing 40000
Processing 50000
Processing 60000
Processing 70000
Processing 80000
Processing 90000
Processing 100000
Number of example in ./data/CBData/cbtest_V_train.txt: 105825
Processing 0
Processing 10000
Processing 20000
Processing 30000
Processing 40000
Processing 50000
Processing 60000
Processing 70000
Processing 80000
Processing 90000
Processing 100000
Number of example in ./data/CBData/cbtest_CN_train.txt: 120769
Processing 0
Processing 10000
Processing 20000
Processing 30000
Processing 40000
Processing 50000
Processing 60000
Processing 70000
Processing 80000
Processing 90000
Processing 100000
Processing 110000
Processing 120000
Number of example in ./data/CBData/cbtest_NE_test_2500ex.txt: 2500
Processing 0
Number of example in ./data/CBData/cbtest_V_valid_2000ex.txt: 2

In [38]:
data['answers']

[89,
 54,
 18,
 93,
 18,
 26,
 18,
 26,
 18,
 28,
 18,
 304,
 168,
 26,
 35,
 54,
 26,
 26,
 21,
 28,
 18,
 93,
 26,
 26,
 1054,
 529,
 449,
 54,
 26,
 54,
 26,
 18,
 18,
 93,
 18,
 93,
 35,
 264,
 54,
 26,
 26,
 255,
 18,
 18,
 26,
 304,
 18,
 54,
 26,
 93,
 304,
 168,
 37,
 54,
 168,
 18,
 54,
 5966,
 54,
 18,
 37,
 28,
 26,
 28,
 304,
 54,
 93,
 93,
 304,
 26,
 1194,
 26,
 26,
 26,
 18,
 93,
 54,
 54,
 304,
 93,
 26,
 93,
 26,
 160,
 26,
 26,
 26,
 57,
 26,
 54,
 26,
 26,
 41,
 304,
 26,
 26,
 54,
 308,
 50,
 304,
 264,
 93,
 103,
 18,
 103,
 26,
 26,
 335,
 26,
 26,
 304,
 54,
 35,
 89,
 18,
 18,
 26,
 26,
 18,
 18,
 35,
 18,
 304,
 37,
 18,
 93,
 18,
 93,
 304,
 18,
 26,
 26,
 304,
 37,
 26,
 18,
 28,
 26,
 54,
 18,
 18,
 236,
 26,
 18,
 26,
 304,
 304,
 26,
 37,
 54,
 26,
 93,
 54,
 18,
 26,
 89,
 21,
 26,
 18,
 304,
 28,
 26,
 93,
 304,
 18,
 26,
 26,
 93,
 35,
 26,
 1021,
 28,
 264,
 54,
 18,
 26,
 54,
 160,
 28,
 26,
 35,
 264,
 35,
 26,
 28,
 168,
 294,
 168,
 168,
 54,
 54,
